In [1]:
!nvidia-smi

Tue Apr 30 13:27:26 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            On   | 00000000:05:00.0 Off |                  N/A |
| 24%   43C    P8    10W / 250W |      1MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            On   | 00000000:06:00.0 Off |                  N/A |
| 23%   41C    P8    10W / 250W |      1MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

### Single GPU

- Single GPU_Setting

In [2]:
import torch

# Change GPU allocatiojn
GPU_NUM = 2 # Choose GPU num
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of cuurent GPU
print('Device:', device)  
print('Count of using GPUs:', torch.cuda.device_count())  
print('Current cuda device:', torch.cuda.current_device()) 

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Device: cuda:2
Count of using GPUs: 4
Current cuda device: 2
TITAN Xp
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


/home/gayeon42/.local/lib/python3.9/site-packages/torch/cuda/memory.py:391: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [3]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 1 (GPU #1 한개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 0 (GPU #1 의미)

Device: cuda
Count of using GPUs: 4
Current cuda device: 2


- Single GPU_Test

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# GPU를 사용할 수 있는지 확인하고 사용 가능하다면 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 임의의 데이터 생성
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32).to(device)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32).to(device)

# 다층 퍼셉트론(MLP) 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # 입력층에서 은닉층으로의 연결
        self.fc2 = nn.Linear(4, 1)  # 은닉층에서 출력층으로의 연결
        self.activation = nn.Sigmoid()  # 활성화 함수: 시그모이드

    def forward(self, x):
        x = self.activation(self.fc1(x))  # 은닉층
        x = self.activation(self.fc2(x))  # 출력층
        return x

# 모델 초기화 및 GPU로 이동
model = MLP().to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()  # 이진 분류를 위한 교차 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass 및 옵티마이저 업데이트
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 학습된 모델 테스트
with torch.no_grad():
    test_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32).to(device)
    predictions = model(test_data)
    predictions = np.round(predictions.cpu().numpy())  # 예측 결과 반올림
    print("예측 결과:")
    print(predictions)

Epoch [100/1000], Loss: 0.6921
Epoch [200/1000], Loss: 0.6591
Epoch [300/1000], Loss: 0.3674
Epoch [400/1000], Loss: 0.1743
Epoch [500/1000], Loss: 0.0921
Epoch [600/1000], Loss: 0.0491
Epoch [700/1000], Loss: 0.0296
Epoch [800/1000], Loss: 0.0201
Epoch [900/1000], Loss: 0.0147
Epoch [1000/1000], Loss: 0.0114
예측 결과:
[[0.]
 [1.]
 [1.]
 [0.]]


### Multi GPU
- Multi GPU Training : 여러 개의 GPU를 사용하여 딥러닝 모델을 학습하는 방법

- Multi GPU_Setting

In [1]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (1,2 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 0 (1,2 중 앞의 GPU #1 의미)

Device: cuda
Count of using GPUs: 2
Current cuda device: 0


- Multi GPU_Test

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# GPU를 사용할 수 있는지 확인하고 사용 가능하다면 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 임의의 데이터 생성
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32).to(device)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32).to(device)

# 다층 퍼셉트론(MLP) 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # 입력층에서 은닉층으로의 연결
        self.fc2 = nn.Linear(4, 1)  # 은닉층에서 출력층으로의 연결
        self.activation = nn.Sigmoid()  # 활성화 함수: 시그모이드

    def forward(self, x):
        x = self.activation(self.fc1(x))  # 은닉층
        x = self.activation(self.fc2(x))  # 출력층
        return x

# 모델 초기화 및 GPU로 이동
model = MLP().to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()  # 이진 분류를 위한 교차 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass 및 옵티마이저 업데이트
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 학습된 모델 테스트
with torch.no_grad():
    test_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32).to(device)
    predictions = model(test_data)
    predictions = np.round(predictions.cpu().numpy())  # 예측 결과 반올림
    print("예측 결과:")
    print(predictions)

Epoch [100/1000], Loss: 0.6166
Epoch [200/1000], Loss: 0.3153
Epoch [300/1000], Loss: 0.1664
Epoch [400/1000], Loss: 0.1032
Epoch [500/1000], Loss: 0.0712
Epoch [600/1000], Loss: 0.0526
Epoch [700/1000], Loss: 0.0407
Epoch [800/1000], Loss: 0.0326
Epoch [900/1000], Loss: 0.0268
Epoch [1000/1000], Loss: 0.0224
예측 결과:
[[0.]
 [1.]
 [1.]
 [0.]]
